In [2]:
import pandas as pd
import pathlib
import numpy as np

## Load Data

In [5]:
raw_data_dir = pathlib.Path("../../data/raw/soroush_2")

trans_res_path = raw_data_dir.joinpath('2023.09.06 BSA_OVA_quant_super_akin.csv')
trans_res_df_raw = pd.read_csv(trans_res_path)
trans_res_df_raw.columns = trans_res_df_raw.columns.map(str.lower)
trans_res_df_raw.head()

,peptide,protein,replicate,precursor mz,precursor charge,product mz,product charge,fragment ion,retention time,area,background
0,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,844.42355,2,844.42355,2,precursor,47.00,7.575379e+09,133355208
1,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D0_column12_2uL_80,844.42355,2,844.42355,2,precursor,45.96,9.467220e+09,98842
2,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D1_column12_2uL_141,844.42355,2,844.42355,2,precursor,46.89,5.926985e+07,0
3,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D1_column12_2uL_159,844.42355,2,844.42355,2,precursor,45.72,3.679658e+09,20224484
4,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D2_column12_2uL_32,844.42355,2,844.42355,2,precursor,46.89,3.454373e+09,30604174


In [6]:

trans_res_df_raw.peptide.unique()

array(['GGLEPINFQTAADQAR', 'AEFVEVTK', 'YICDNQDTISSK'], dtype=object)

## Compute Area ratio

In [7]:
# def grp_area_ratio(grouped_df:pd.DataFrame):
    
#     """compute the ratio of areas with higer precursor mz value as numerator."""

#     df = grouped_df[['precursor mz', 'area']].copy()
#     df = df.sort_values(by='precursor mz', ascending=True)
#     area_ratio = df['area'].iloc[1] / df['area'].iloc[0]

#     return area_ratio

def grp_area_ratio_n_area_cols(grouped_df:pd.DataFrame):
    
    """compute the ratio of areas with higer precursor mz value as numerator."""

    df = grouped_df[['precursor mz', 'area']].copy()
    df = df.sort_values(by='precursor mz', ascending=True)
    area_ratio = df['area'].iloc[1] / df['area'].iloc[0]
    area_min, area_max  =  df['area'].iloc[0], df['area'].iloc[1]

    col_vals = {'area_ratio': area_ratio, 'area_min':area_min, 'area_max':area_max}

    return pd.Series(col_vals)

cols_to_group_by = ['peptide', 'protein', 'replicate', 'product charge', 'fragment ion']
# df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio).reset_index(name='area_ratio')
df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio_n_area_cols).reset_index()


# df_area_ratio = df_area_ratio[df_area_ratio['fragment ion'] == 'precursor']  # select only precursor
print(df_area_ratio.shape)
df_area_ratio.head()

(1080, 8)


c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0


## Get peptide Dilution Conc.

In [27]:
df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Soroush_2023_09_06_peptide_dilution_conc.csv'))
# df_peptide_dilution_conc.columns


df_peptide_dilution_conc.head()
raw_dilution_col_names = df_peptide_dilution_conc.columns[1:]


df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='Peptides', value_vars=raw_dilution_col_names, 
                                                        var_name='dilution', value_name='heavy_conc')
df_peptide_dilution_conc['dilution'] = df_peptide_dilution_conc['dilution'].apply(lambda x: x[:2])
df_peptide_dilution_conc['peptide_dilution_name'] = df_peptide_dilution_conc['Peptides'] + '_' + \
df_peptide_dilution_conc['dilution'] 
df_peptide_dilution_conc.head()

,Peptides,dilution,heavy_conc,peptide_dilution_name
0,GGLEPINFQTAADQAR,D1,0.736000,GGLEPINFQTAADQAR_D1
1,AEFVEVTK,D1,0.403667,AEFVEVTK_D1
2,YICDNQDTISSK,D1,1.257333,YICDNQDTISSK_D1
3,GGLEPINFQTAADQAR,D2,0.184000,GGLEPINFQTAADQAR_D2
4,AEFVEVTK,D2,0.100917,AEFVEVTK_D2


In [28]:



def get_peptide_dilution_comb(row):
    """function to create a new column from peptide and dilution name."""
    peptide = row['peptide']
    replicate  = row['replicate']
    dilution = get_dilution_from_replicate(rep_string=replicate)

    return peptide + '_' + dilution

df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Soroush_2023_09_06_peptide_dilution_conc.csv'))
raw_dilution_col_names = df_peptide_dilution_conc.columns[1:]

df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='Peptides', value_vars=raw_dilution_col_names, 
                                                        var_name='dilution', value_name='heavy_conc')
df_peptide_dilution_conc['dilution'] = df_peptide_dilution_conc['dilution'].apply(lambda x: x[:2])
peptide_dilution_name_col = 'peptide_dilution_name'
df_peptide_dilution_conc[peptide_dilution_name_col] = df_peptide_dilution_conc['Peptides'] + '_' + \
                                                    df_peptide_dilution_conc['dilution'] 
df_peptide_dilution_conc.head()


,Peptides,dilution,heavy_conc,peptide_dilution_name
0,GGLEPINFQTAADQAR,D1,0.736000,GGLEPINFQTAADQAR_D1
1,AEFVEVTK,D1,0.403667,AEFVEVTK_D1
2,YICDNQDTISSK,D1,1.257333,YICDNQDTISSK_D1
3,GGLEPINFQTAADQAR,D2,0.184000,GGLEPINFQTAADQAR_D2
4,AEFVEVTK,D2,0.100917,AEFVEVTK_D2


### Merge data

In [38]:
def get_dilution_from_replicate(rep_string:str):
    """Get the dilution symbol from the replicate name."""

    str_to_check = rep_string.split('_')[4]
    if str_to_check.startswith('col'):
        dilution = 'D0'
    else:
        dilution = str_to_check

    return dilution
df_area_ratio[peptide_dilution_name_col] = df_area_ratio.apply(get_peptide_dilution_comb, axis=1)



df_area_ratio_conc = pd.merge(df_area_ratio, right=df_peptide_dilution_conc, how='left', 
                              on=peptide_dilution_name_col, suffixes=('', '_y'))
df_area_ratio_conc.drop(df_area_ratio_conc.filter(regex='_y$').columns, axis=1, inplace=True)

# def get_rep_plot_cat(rep_val:str):
#     """Get the plot category (C18_SDS, C18_SDC, etc..)."""

#     select_elements = rep_val.split('_')[1:3]

#     cat = '_'.join(select_elements)
#     return cat

def get_rep_plot_cat(row):
    """Get the plot category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']
    fragment_ion = row['fragment ion']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide] + [fragment_ion] + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

# create plot categories
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc.apply(get_rep_plot_cat, axis=1)
df_area_ratio_conc.head()
print(df_area_ratio_conc.isna().sum())
df_area_ratio_conc = df_area_ratio_conc[df_area_ratio_conc['area_ratio'].notna()]
print(df_area_ratio_conc.isna().sum())

peptide                  0
protein                  0
replicate                0
product charge           0
fragment ion             0
area_ratio               2
area_min                 0
area_max                 0
peptide_dilution_name    0
Peptides                 0
dilution                 0
heavy_conc               0
plot_cat                 0
dtype: int64
peptide                  0
protein                  0
replicate                0
product charge           0
fragment ion             0
area_ratio               0
area_min                 0
area_max                 0
peptide_dilution_name    0
Peptides                 0
dilution                 0
heavy_conc               0
plot_cat                 0
dtype: int64


In [ ]:
df_area_ratio

## Get Linear Fit parameters

In [39]:
from sklearn import linear_model, metrics
import matplotlib.pyplot as plt

def get_linear_fit(data:pd.DataFrame, plot_cat:str):

    cat_data = data[data['plot_cat'] == plot_cat]
    # print(cat_data)

    # fig, ax = plt.subplots()

    x = cat_data['area_ratio'].values
    y = cat_data['heavy_conc'].values

    # ax.scatter(x, y)

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    model = linear_model.LinearRegression()

    model.fit(x, y)

    y_fit = model.predict(x)

    # ax.plot(x.flatten(), y_fit.flatten())

    r2 = metrics.r2_score(y, y_fit)
    intercept = model.intercept_.squeeze()
    grad = model.coef_[0].squeeze()


    # print(f"R^2: {r2}")
    # print(f"intercept: {intercept}")
    # print(f"gradient:{grad}")

    # plt.show()
    
    return (r2, intercept, grad)

plot_cats = df_area_ratio_conc['plot_cat'].unique()

# plot_df = df_area_ratio_conc[df_area_ratio_conc.peptide=='GGLEPINFQTAADQAR'] # get only one peptide
# plot_df = plot_df[[peptide_dilution_name_col, 'area_ratio', 'heavy_conc', 'plot_cat']]
plot_df = df_area_ratio_conc


cats = []
r2s = []
intercepts = []
grads = []

for cat in plot_cats:
    # print(cat)
    r2, intercept, grad = get_linear_fit(plot_df, cat)
    cats.append(cat)
    r2s.append(r2)
    intercepts.append(intercept)
    grads.append(grad)
    



df_plot_cat_fit_params = pd.DataFrame({'plot_cat':cats,
              'R2':r2s,
              'intercept':intercepts,
              'gradient':grads})




ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [8]:
df_area_ratio_conc_n_fit_params = pd.merge(df_area_ratio_conc,
                                           right=df_plot_cat_fit_params,
                                           on='plot_cat',
                                           how='left')

df_area_ratio_conc_n_fit_params
# save to csv file
# df_area_ratio_conc_n_fit_params.to_csv('area_ratio_conc_n_fit_params.csv', index=False)
    

# Fit metric

In [12]:
def get_fit_param_cat(row):
    """Get the fit param category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide]  + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def qtest(data, right=True):

    sorted_data = sorted(data)

    if right:
        gap = sorted_data[-1] - sorted_data[-2]

    else:
        gap = sorted_data[1] - sorted_data[0]

    try:
        range = sorted_data[-1] - sorted_data[0]
        q_val = gap / range
    except ZeroDivisionError:
        q_val = gap

    return q_val

def get_grp_fit_agg(grouped_df:pd.DataFrame):
    """Get the aggregate values for each group."""

    df = grouped_df[['intercept', 'gradient', 'R2']].copy()
    gradient = df['gradient'].values
    intercept = df['intercept'].values
    r2 = df['R2'].values

    mean_grad = gradient.mean()
    stdv_grad = gradient.std()
    cov_grad = stdv_grad / mean_grad
    qtest_grad_right = qtest(gradient, right=True)

    mean_intercept = intercept.mean()
    stdv_intercept = intercept.std()
    cov_intercept = stdv_intercept / mean_intercept
    qtest_intercept_right = qtest(intercept, right=True)

    mean_r2 = r2.mean()
    stdv_r2 = r2.std()
    cov_r2 = stdv_r2 / mean_r2
    qtest_r2_right = qtest(r2, right=True)

    cols_dict = {'mean_grad':mean_grad, 'stdv_grad':stdv_grad,
                 'cov_grad':cov_grad, 'qtest_grad':qtest_grad_right, 
                 'mean_intercept':mean_intercept, 'stdv_intercept':stdv_intercept,
                 'cov_intercept':cov_intercept, 'qtest_intercept':qtest_intercept_right,
                 'mean_r2':mean_r2, 'stdv_r2':stdv_r2,
                 'cov_r2':cov_r2, 'qtest_r2':qtest_r2_right
                 }

    return pd.Series(cols_dict)


# create plot categories
temp = df_area_ratio_conc_n_fit_params.copy()
temp['fit_param_grp'] = temp.apply(get_fit_param_cat, axis=1)

cols_to_group_by = ['fit_param_grp']
df_fit_param_agg = temp.groupby(cols_to_group_by).apply(get_grp_fit_agg).reset_index()

df_w_fit_param_agg = pd.merge(left=temp, right=df_fit_param_agg, how='left', on=cols_to_group_by)
df_w_fit_param_agg.head(3)

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,heavy_conc,...,cov_grad,qtest_grad,mean_intercept,stdv_intercept,cov_intercept,qtest_intercept,mean_r2,stdv_r2,cov_r2,qtest_r2
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,b2,4.785271,425566400.0,2.036451e+09,AEFVEVTK_D2,86.5,...,0.050112,0.0,-1.424813,0.435773,-0.305846,0.0,0.999068,0.000534,0.000534,0.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y2,5.214290,52236184.0,2.723746e+08,AEFVEVTK_D2,86.5,...,0.050112,0.0,-1.424813,0.435773,-0.305846,0.0,0.999068,0.000534,0.000534,0.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y4,4.992701,92338456.0,4.610183e+08,AEFVEVTK_D2,86.5,...,0.050112,0.0,-1.424813,0.435773,-0.305846,0.0,0.999068,0.000534,0.000534,0.0


In [13]:
df_w_fit_param_agg.to_csv('data_with_fit_param_aggregate.csv', index=False)